# Test of first, basic model on full dataset

In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import h5py

import tensorflow as tf
# Set logging level to avoid unnecessary messages
tf.get_logger().setLevel('ERROR')
# Set autograph verbosity to avoid unnecessary messages
tf.autograph.set_verbosity(0)

import keras
import tensorflow_io as tfio
import tensorflow_probability as tfp
import tensorflow_extra as tfe

import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio

from pathlib import Path
import sys
import os
import time
import warnings
# suppress all warnings
warnings.filterwarnings("ignore")

XC_ROOTDIR = '../../data/' # directory to save data in
XC_DIR = 'test_dataset10' # subdirectory name of dataset

tf.config.set_visible_devices([], 'GPU')

2024-07-04 23:46:27.151970: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 23:46:27.152026: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 23:46:27.153546: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-04 23:46:27.162088: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-04 23:46:28.084636: W tensorflow/compiler/tf2

In [2]:
import wandb

### Load data

In [3]:
df = pd.read_csv("../../data/dataset_train.csv")
len(df)

13875

## Configurations

In [4]:
class cfg:
    # random seed
    seed = 42

    # audio clip settings
    sr = 22050
    duration = 15 # the duration of the clips
    
    n_samples = duration*sr
    
    hop_length = 2048 # "stepsize" of the fft for the melspectrograms
    nfft = 4096 # windowsize of the fft for the melspectrograms
    n_mels = 128 # number of mel frequency bins
    fmax = sr/2 # maximum frequency in the melspectrograms
    input_dim = (n_mels, int(duration*sr//hop_length + 1))
    
    # training settings
    batch_size = 32
    n_epochs = 50
    
    # class labels/names
    names = list(np.unique(df.en))
    n_classes = len(names)
    labels = list(range(n_classes))
    label2name = dict(zip(labels, names))
    name2label = {v:k for k,v in label2name.items()}

# set random seed in keras
tf.keras.utils.set_random_seed(cfg.seed)

### Up- and downsample df

In [5]:
def upsample_data(df, thr=200):
    # get the class distribution
    class_dist = df['en'].value_counts()

    # identify the classes that have less than the threshold number of samples
    down_classes = class_dist[class_dist < thr].index.tolist()

    # create an empty list to store the upsampled dataframes
    up_dfs = []

    # loop through the undersampled classes and upsample them
    for c in down_classes:
        # get the dataframe for the current class
        class_df = df.query("en==@c")
        # find number of samples to add
        num_up = thr - class_df.shape[0]
        # upsample the dataframe
        class_df = class_df.sample(n=num_up, replace=True, random_state=cfg.seed)
        # append the upsampled dataframe to the list
        up_dfs.append(class_df)

    # concatenate the upsampled dataframes and the original dataframe
    up_df = pd.concat([df] + up_dfs, axis=0, ignore_index=True)
    
    return up_df

def downsample_data(df, thr=400):
    # get the class distribution
    class_dist = df['en'].value_counts()
    
    # identify the classes that have less than the threshold number of samples
    up_classes = class_dist[class_dist > thr].index.tolist()

    # create an empty list to store the upsampled dataframes
    down_dfs = []

    # loop through the undersampled classes and upsample them
    for c in up_classes:
        # get the dataframe for the current class
        class_df = df.query("en==@c")
        # Remove that class data
        df = df.query("en!=@c")
        # upsample the dataframe
        class_df = class_df.sample(n=thr, replace=False, random_state=cfg.seed)
        # append the upsampled dataframe to the list
        down_dfs.append(class_df)

    # concatenate the upsampled dataframes and the original dataframe
    down_df = pd.concat([df] + down_dfs, axis=0, ignore_index=True)
    
    return down_df

In [6]:
df = upsample_data(downsample_data(df, thr=100), thr=100)

In [7]:
len(df)

4600

In [8]:
df["label"] = None

for idx in tqdm(df.index):
    df.loc[idx, "label"] = cfg.name2label[df.loc[idx, "en"]]

100%|█████████████████████████████████████████████████████████████████████████████| 4600/4600 [00:00<00:00, 5930.79it/s]


In [9]:
df.fullfilename = "../" + df.fullfilename

### Functions, data generator etc.

In [10]:
# Generates random integer # from https://www.kaggle.com/code/wengsilu/birdclef24pretraining
def random_int(shape=[], minval=0, maxval=1):
    return tf.random.uniform(shape=shape, minval=minval, maxval=maxval, dtype=tf.int32)

# Generats random float
def random_float(shape=[], minval=0.0, maxval=1.0):
    rnd = tf.random.uniform(shape=shape, minval=minval, maxval=maxval, dtype=tf.float32)
    return rnd

def load_spectrogram_slice(hdf5_path, name, start_row = 0, end_row =None, start_col = 0, end_col = None):
    with h5py.File(hdf5_path, 'r') as f:
        spectrogram_slice = f[name][start_row:end_row, start_col:end_col]
    return spectrogram_slice

def load_random_spec_slice(df, ID):
    name = df.spectrogram.iloc[ID]
    hdf5_path = os.path.dirname(df.fullfilename.iloc[ID]) + "/spectrograms.h5"
    spec_length = df.length_spectrogram.iloc[ID]
    if spec_length > cfg.input_dim[1]:
        rdm = random_int(maxval= spec_length - cfg.input_dim[1])
        return load_spectrogram_slice(hdf5_path = hdf5_path, name = name, start_col = rdm, end_col = rdm + cfg.input_dim[1])
    else:
        return load_spectrogram_slice(hdf5_path = hdf5_path, name = name)

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, dataframe,
                 batch_size=cfg.batch_size, 
                 dim=cfg.input_dim,
                 n_channels =  1,
                 n_classes=cfg.n_classes, 
                 shuffle=True
                ):
        'Initialization'
        self.dim = dim
        self.n_channels = n_channels
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.dataframe = dataframe
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = load_random_spec_slice(self.dataframe, ID).reshape(*self.dim, self.n_channels)
            # Store class
            y[i] = cfg.name2label[df.en.iloc[ID]]
        X = X.reshape(len(X), *self.dim, self.n_channels)
        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

def plot_history(network_history):
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(network_history.history['loss'])
    plt.plot(network_history.history['val_loss'])
    plt.legend(['Training', 'Validation'])

    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.plot(network_history.history['accuracy'])
    plt.plot(network_history.history['val_accuracy'])
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()

def pad_spectrogram(spec, shape = cfg.input_dim, random = False):
    _ = np.zeros(shape)
    if random:
        rdm = random_int(maxval=shape[1]-spec.shape[1])
        _[:,rdm: rdm + spec.shape[1]] = spec 
    else:
        _[:,:spec.shape[1]] = spec
    return _

def predict_file(df, ID, model):
    name = df.spectrogram.iloc[ID]
    hdf5_path = os.path.dirname(df.fullfilename.iloc[ID]) + "/spectrograms.h5"
    spec_length = df.length_spectrogram.iloc[ID]
    spec = load_spectrogram_slice(hdf5_path, name)
    slices = []
    
    for i in range(spec_length//cfg.input_dim[1]):
        slices.append(spec[:,i*cfg.input_dim[1]:(i+1)*cfg.input_dim[1]])
    if spec_length%cfg.input_dim[1]/cfg.input_dim[1] > 5/cfg.duration:
        # consider last slice, only if it is longer than the shortest clips in the dataset 
        slices.append(pad_spectrogram(spec[:, (i+1)*cfg.input_dim[1]:None], random = True))
    
    preds = model.predict(np.expand_dims(np.array(slices), axis = -1))
    
    return np.mean(preds, axis = 0) # return mean prediction

## Initialize wandb

In [11]:
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbEvalCallback

run = wandb.init(
    # Set the project where this run will be logged
    project="CNN_Birdcall_classification",
    # Track hyperparameters and run metadata
    config={
        "batch_size": cfg.batch_size,
        "epochs": cfg.n_epochs,
        "n_classes": cfg.n_classes,
        "audio duration": cfg.duration
    },
)

wandb: Currently logged in as: cfr (ML_Project2024TUD). Use `wandb login --relogin` to force relogin


In [12]:
#wandb_metric = WandbMetricsLogger()

path = "../models/Basemodel_test0.h5"

model_checkpoint = WandbModelCheckpoint(path, 
                                        monitor = "val_accuracy", 
                                        verbose = 1, 
                                        save_best_only=True,
                                        mode = "max"
                                       )



wandb: WARNING When using `save_best_only`, ensure that the `filepath` argument contains formatting placeholders like `{epoch:02d}` or `{batch:02d}`. This ensures correct interpretation of the logged artifacts.


In [13]:
from keras.callbacks import EarlyStopping, CSVLogger #, ModelCheckpoint
#path = "../models/Basemodel_test0.h5"

#checkpoint = ModelCheckpoint(path, 
#                             monitor = "val_accuracy", 
#                             verbose = 1, 
#                             save_best_only=True,
#                             mode = "max"
#                            )

early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=6,
    verbose=0,
    mode="min",
    restore_best_weights=False,
    start_from_epoch=10,
)

log_file = "../models/Basemodel_test0_log.csv"

csv_log = CSVLogger(log_file, separator=",", append=True)

In [14]:
callbacks = [model_checkpoint, csv_log, WandbMetricsLogger()]

## Build model

In [15]:
from tensorflow.keras.layers import Layer, Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Input

zscore_layer = tfe.layers.ZScoreMinMax()

def build_model():
    inp = Input(shape=(*cfg.input_dim, 1))
    
    # Normalize
    x = zscore_layer(inp)
    
    # Base model
    x = Conv2D(32, kernel_size=(3, 3), padding='valid', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2), padding="valid")(x)
    x = Conv2D(64, kernel_size=(3, 3), padding='valid', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2), padding="valid")(x)
    x = Dropout(0.4)(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(32, activation='relu')(x)
    output = Dense(cfg.n_classes, activation='softmax')(x)
    
    model = tf.keras.models.Model(inputs=inp, outputs=output, name = "Basemodel")
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [16]:
model = build_model()

Model: "Basemodel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 162, 1)]     0         
                                                                 
 z_score_min_max (ZScoreMin  (None, 128, 162, 1)       0         
 Max)                                                            
                                                                 
 conv2d (Conv2D)             (None, 126, 160, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 80, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 78, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 39, 64)        0 

## Train validation split

In [17]:
from sklearn.model_selection import train_test_split

id_train, id_val, y_train, y_val = train_test_split(range(len(df)), df["en"].to_list(), test_size = 0.3, random_state = cfg.seed)

training_generator = DataGenerator(id_train, y_train, df)
validation_generator = DataGenerator(id_val, y_val, df)

In [18]:
hist = model.fit_generator(generator=training_generator,
                           validation_data=validation_generator,
                           use_multiprocessing=True, 
                           verbose = 2, 
                           epochs = cfg.n_epochs,
                           callbacks = callbacks
                          )

Epoch 1/50


2024-07-04 23:46:41.964609: I external/local_xla/xla/service/service.cc:168] XLA service 0x5611e046b5d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-07-04 23:46:41.964652: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-07-04 23:46:41.977486: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1720129602.040735    2606 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-07-04 23:46:42.042858: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-07-04 23:46:42.045201: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-07-04 23:46:42.471359: E external/local_xla/xl


Epoch 1: val_accuracy improved from -inf to 0.01817, saving model to ../models/Basemodel_test0.h5
100/100 - 69s - loss: 3.8326 - accuracy: 0.0181 - val_loss: 3.8290 - val_accuracy: 0.0182 - 69s/epoch - 692ms/step
Epoch 2/50

Epoch 2: val_accuracy improved from 0.01817 to 0.02544, saving model to ../models/Basemodel_test0.h5
100/100 - 66s - loss: 3.8295 - accuracy: 0.0237 - val_loss: 3.8286 - val_accuracy: 0.0254 - 66s/epoch - 656ms/step
Epoch 3/50

Epoch 3: val_accuracy did not improve from 0.02544
100/100 - 70s - loss: 3.8261 - accuracy: 0.0291 - val_loss: 3.8231 - val_accuracy: 0.0174 - 70s/epoch - 700ms/step
Epoch 4/50

Epoch 4: val_accuracy did not improve from 0.02544
100/100 - 64s - loss: 3.8215 - accuracy: 0.0294 - val_loss: 3.8210 - val_accuracy: 0.0240 - 64s/epoch - 640ms/step
Epoch 5/50

Epoch 5: val_accuracy improved from 0.02544 to 0.07267, saving model to ../models/Basemodel_test0.h5
100/100 - 72s - loss: 3.7464 - accuracy: 0.0472 - val_loss: 3.5419 - val_accuracy: 0.0727

2024-07-05 00:25:15.446435: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.



Epoch 34: val_accuracy did not improve from 0.57267
100/100 - 63s - loss: 1.3381 - accuracy: 0.6228 - val_loss: 1.6955 - val_accuracy: 0.5574 - 63s/epoch - 632ms/step
Epoch 35/50

Epoch 35: val_accuracy improved from 0.57267 to 0.58794, saving model to ../models/Basemodel_test0.h5
100/100 - 64s - loss: 1.3429 - accuracy: 0.6156 - val_loss: 1.6371 - val_accuracy: 0.5879 - 64s/epoch - 637ms/step
Epoch 36/50

Epoch 36: val_accuracy did not improve from 0.58794
100/100 - 64s - loss: 1.3131 - accuracy: 0.6206 - val_loss: 1.6876 - val_accuracy: 0.5705 - 64s/epoch - 644ms/step
Epoch 37/50

Epoch 37: val_accuracy improved from 0.58794 to 0.59012, saving model to ../models/Basemodel_test0.h5
100/100 - 64s - loss: 1.3297 - accuracy: 0.6266 - val_loss: 1.6209 - val_accuracy: 0.5901 - 64s/epoch - 638ms/step
Epoch 38/50

Epoch 38: val_accuracy did not improve from 0.59012
100/100 - 65s - loss: 1.2885 - accuracy: 0.6319 - val_loss: 1.6671 - val_accuracy: 0.5821 - 65s/epoch - 646ms/step
Epoch 39/50


In [19]:
run.finish()

epoch/accuracy,▁▁▁▁▂▃▃▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,████▇▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▁▁▁▁▃▃▄▄▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
epoch/val_loss,████▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,0.65344
epoch/epoch,49
epoch/learning_rate,0.001
epoch/loss,1.18925
epoch/val_accuracy,0.60756
